In [ ]:
### SETUP ###
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import PIL
import PIL.Image
import tensorflow_datasets as tfds

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### LOAD DATA USING TF.KERAS ###
# batch_size = 32
# img_height = 180
# img_width = 180
dirname = '/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset'
builder = tfds.builder('celebs', data_dir=dirname)
